In [1]:
import os
os.listdir()

['check.py', 'models', 'Training', 'training.ipynb']

In [2]:
import pandas as pd
df=pd.read_csv('Training/Dataset/synthetic_logs.csv')
df

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
...,...,...,...,...,...
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert
2406,1/11/2025 5:32,ModernHR,User 3844 account experienced multiple failed ...,Security Alert,bert
2407,2025-08-03 03:07:47,ThirdPartyAPI,nova.metadata.wsgi.server [req-b6d4a270-accb-4...,HTTP Status,bert
2408,11/11/2025 11:52,BillingSystem,Email service affected by failed transmission,Critical Error,bert


In [3]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [4]:
df[df.target_label=='HTTP Status']

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert
...,...,...,...,...,...
2399,2025-03-08 06:23:00,ThirdPartyAPI,nova.metadata.wsgi.server [req-ba29717b-249a-4...,HTTP Status,bert
2401,2025-12-05 15:51:51,ModernCRM,nova.osapi_compute.wsgi.server [req-4bdf00b0-3...,HTTP Status,bert
2404,2025-09-18 02:18:30,ThirdPartyAPI,nova.osapi_compute.wsgi.server [req-2c9c783f-3...,HTTP Status,bert
2405,2025-08-13 07:29:25,ModernHR,nova.osapi_compute.wsgi.server [req-96c3ec98-2...,HTTP Status,bert


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get log messages (assuming column name is 'log_message')
log_messages = df['log_message'].astype(str).tolist()

# Generate embeddings
embeddings = model.encode(log_messages, show_progress_bar=True)

In [ ]:
embeddings

In [ ]:
dbscan=DBSCAN(eps=0.2, min_samples=1, metric='cosine')
clusters=dbscan.fit_predict(embeddings)
df['cluster'] = clusters
df

In [ ]:
df[df['cluster']==0]

In [ ]:
df['cluster'].unique()

In [ ]:

cluster_counts = df['cluster'].value_counts()
large_clusters = cluster_counts[cluster_counts >= 10].index.tolist()

for cluster_id in large_clusters:
    print(f"\nCluster {cluster_id} (size: {cluster_counts[cluster_id]}):")
    messages = df[df['cluster'] == cluster_id]['log_message'].head(5)
    for msg in messages:
        print(msg)

In [ ]:
import re
def classify_with_regex(log_messages):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Account with ID .* created by *": "User Action",
    }
    for pattern,label in regex_patterns.items():
        if re.search(pattern, log_messages, re.IGNORECASE):
            return label
    return None

In [ ]:
classify_with_regex("User User225 logged in.")

In [ ]:
df['regex_label']=df['log_message'].apply(classify_with_regex)
df

In [ ]:
df[df.regex_label.notnull()]

In [ ]:
df_non_regex=df[df.regex_label.isnull()]
df_non_regex

In [ ]:
label_counts = df_non_regex['target_label'].value_counts()
small_labels = label_counts[label_counts < 10].index.tolist()

for label in small_labels:
    print(f"\nTarget label: {label} (count: {label_counts[label]})")
    messages = df_non_regex[df_non_regex['target_label'] == label]['source']+"__"+df_non_regex[df_non_regex['target_label'] == label]['log_message']
    for msg in messages:
        print(msg)

In [ ]:
df_non_legacy_CRM=df_non_regex[df_non_regex.source!='LegacyCRM']
df_non_legacy_CRM

In [ ]:
embeddings_non_legacy = model.encode(df_non_legacy_CRM['log_message'].astype(str).tolist(), show_progress_bar=True)
embeddings_non_legacy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Prepare features and labels
X = embeddings_non_legacy
y = df_non_legacy_CRM['target_label'].values

# Split data (stratified, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Fit logistic regression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Predict and show classification report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import os
import joblib

# Create 'models' folder if it doesn't exist
os.makedirs('models', exist_ok=True)

# Save the trained logistic regression model
joblib.dump(clf, 'models/logg_classifier.joblib')